In [15]:
from groq import Groq

# client = Groq(api_key="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")

In [16]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma

In [17]:
from langchain_core.documents import Document

In [18]:
loader = PyPDFLoader("RAG_Crash_Course_Notes.pdf")
documents = loader.load()

In [19]:
documents

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-02-09T10:23:18+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-09T10:23:18+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'RAG_Crash_Course_Notes.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Retrieval Augmented Generation (RAG) Crash Course\nNotes\nAuthor: Saniya Shaik\nPurpose: Learning RAG concepts, ingestion, embeddings, and retrieval\nLevel: Beginner to Intermediate\nYear: 2026'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-02-09T10:23:18+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-09T10:23:18+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'RAG_Crash_Course_Notes.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, pag

### Converting text into Chunks

In [20]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

In [21]:
chunks = splitter.split_documents(documents)
print(len(documents),len(chunks))

2 5


In [22]:
chunks[1]

Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-02-09T10:23:18+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-02-09T10:23:18+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'RAG_Crash_Course_Notes.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content='1. Introduction to Large Language Models (LLMs)\nLarge Language Models (LLMs) are deep learning models trained on massive amounts of text data.\nThey learn patterns, grammar, context, and semantics of language using probability distributions.\n2. What is Retrieval Augmented Generation (RAG)?\nRetrieval Augmented Generation (RAG) enhances LLMs by retrieving external knowledge before\ngenerating answers.\n3. Why RAG is Needed')

In [23]:
chunks[1].metadata

{'producer': 'ReportLab PDF Library - www.reportlab.com',
 'creator': '(unspecified)',
 'creationdate': '2026-02-09T10:23:18+00:00',
 'author': '(anonymous)',
 'keywords': '',
 'moddate': '2026-02-09T10:23:18+00:00',
 'subject': '(unspecified)',
 'title': '(anonymous)',
 'trapped': '/False',
 'source': 'RAG_Crash_Course_Notes.pdf',
 'total_pages': 2,
 'page': 1,
 'page_label': '2'}

**Embeddings**

In [24]:
from sentence_transformers import SentenceTransformer
import numpy as np

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embedding(text):
    return embed_model.encode(text, convert_to_numpy=True)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 272.05it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [25]:
embeddings = np.array(
    [get_embedding(chunk.page_content) for chunk in chunks]
).astype("float32")

#### vectorStoreDB

In [26]:
import faiss

dimension = embeddings.shape[1]  #no of dimensions
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [27]:
def retrieve(query, k=3):
    query_vector = get_embedding(query).reshape(1, -1).astype("float32")
    distances, indices = index.search(query_vector, k)

    retrieved_chunks = [chunks[i].page_content for i in indices[0]]
    return retrieved_chunks

In [28]:
from groq import Groq

client = Groq(api_key="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")

def ask_groq(question):
    context = retrieve(question)

    prompt = f"""
    Answer using ONLY the context below.
    If the answer is not in the context, say "Not found in document."

    Context:
    {context}

    Question:
    {question}
    """

    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You answer only from provided context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return completion.choices[0].message.content

In [29]:
query = "Data Ingestion in RAG?"
print(ask_groq(query))

Data Ingestion in RAG prepares raw data for retrieval by loading, cleaning, chunking, embedding, and storing.
